<a href="https://colab.research.google.com/github/naraquev/Private-AI/blob/master/Week5/Differential_Privacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
from IPython.display import clear_output

In [0]:
class teacher():
  def __init__(self, dataL, device, batch_size = 32, epochs=5, print_every=120):
    self.dataLoader = dataL
    self.model = t_model()
    self.criterion = nn.NLLLoss()
    self.optimizer = optim.Adam(self.model.parameters(), lr=0.003)
    self.epochs = epochs
    self.dbug = print_every
    self.device =  device
                                           
                                           
                                           
  def train(self):
    self.model.to(self.device)
    #print('Aqui voy!')
    steps = 0
    running_loss = 0
    accuracy = 0
    for e in range(self.epochs):
        # Model in training mode, dropout is on
        self.model.train()
        accuracy=0
        running_loss = 0
        for images, labels in self.dataLoader:
            
            images, labels = images.to(self.device), labels.to(self.device)
            steps += 1
            
            self.optimizer.zero_grad()
            
            output = self.model.forward(images)
            loss = self.criterion(output, labels)
            loss.backward()
            self.optimizer.step()            
            running_loss += loss.item()          
            ps = torch.exp(output)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor))
        #if(e == self.epochs -1 or e==0):
         #   print("Epoch: {}/{}.. ".format(e+1, self.epochs),
          #    "Training Loss: {:.3f}.. ".format(running_loss/len(self.dataLoader)),              
          #    "Train Accuracy: {:.3f}".format(accuracy/len(self.dataLoader)))
    return accuracy/len(self.dataLoader)
    
  def eval(self, dataLoader):  
    outputs = torch.zeros(0, dtype=torch.long).to(self.device)
    self.model.to(self.device)
    # Model in test mode, dropout is off
    self.model.eval()
    result=[]
    for images, labels in dataLoader:
        images, labels = images.to(self.device), labels.to(self.device)
        output = self.model.forward(images)
        ps = torch.argmax(torch.exp(output), dim=1)
        outputs = torch.cat((outputs, ps))
    return outputs
  
  
  
class t_model(nn.Module):
    def __init__(self):
        super(t_model, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [0]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, ), (0.5,))])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
device =  torch.device("cuda:0""cuda:0" if torch.cuda.is_available() else "cpu")
num_teachers = 100
train_len = len(mnist_trainset)
test_len = len(mnist_testset)

In [0]:
def get_samples(num_teachers):
  tam = len(mnist_trainset)
  split= int(tam/num_teachers)
  split_ini = split
  indices = list(range(tam))
  init=0
  samples = []
  for i in range(num_teachers):     
    t_idx = indices[init:split]
    t_sampler = SubsetRandomSampler(t_idx)
    samples.append(t_sampler)
    init = split
    split = split+split_ini
  return samples
def create_teachers(samples):
  teachers = []
  for sample in samples:
    loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=32, sampler=sample)
    t = teacher(loader, device)
    teachers.append(t)    
  return teachers

In [0]:
samples = get_samples(num_teachers)
teachers = create_teachers(samples)

In [0]:
def train_teachers(teachers):
  accuracy = []
  for key, teacher in enumerate(teachers):
    accuracy.append(teacher.train())
    clear_output()
    print('Teacher ', key)
  return accuracy


In [22]:
ac = train_teachers(teachers)
print('The accuracy mean of all teachers is: ', np.mean(ac))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Teacher  0
Teacher  1
Teacher  2
Teacher  3
Teacher  4
Teacher  5
Teacher  6
Teacher  7
Teacher  8
Teacher  9
Teacher  10
Teacher  11
Teacher  12
Teacher  13
Teacher  14
Teacher  15
Teacher  16
Teacher  17
Teacher  18
Teacher  19
Teacher  20
Teacher  21
Teacher  22
Teacher  23
Teacher  24
Teacher  25
Teacher  26
Teacher  27
Teacher  28
Teacher  29
Teacher  30
Teacher  31
Teacher  32
Teacher  33
Teacher  34
Teacher  35
Teacher  36
Teacher  37
Teacher  38
Teacher  39
Teacher  40
Teacher  41
Teacher  42
Teacher  43
Teacher  44
Teacher  45
Teacher  46
Teacher  47
Teacher  48
Teacher  49
Teacher  50
Teacher  51
Teacher  52
Teacher  53
Teacher  54
Teacher  55
Teacher  56
Teacher  57
Teacher  58
Teacher  59
Teacher  60
Teacher  61
Teacher  62
Teacher  63
Teacher  64
Teacher  65
Teacher  66
Teacher  67
Teacher  68
Teacher  69
Teacher  70
Teacher  71
Teacher  72
Teacher  73
Teacher  74
Teacher  75
Teacher  76
Teacher  77
Teacher  78
Teacher  79
Teacher  80
Teacher  81
Teacher  82
Teacher  83
Te

In [0]:
epsilon = 0.2
def eval_data(teachers, mnist_testset):
    preds = torch.torch.zeros((num_teachers, test_len), dtype=torch.long)
    loader = torch.utils.data.DataLoader(mnist_testset, batch_size=32)
    for key, teacher in enumerate(teachers):
      clear_output()
      print('Teacher:', key)      
      result = teacher.eval(loader)
      preds[key] = result
    return preds.numpy()
def mechanism(preds, epsilon=0.2):  
  beta = 1 / epsilon
  labels = np.array([]).astype(int)
  for image_preds in np.transpose(preds):
    label_counts = np.bincount(image_preds, minlength=10)
    ori_label_counts = np.bincount(image_preds, minlength=10)
    for i in range(10):
      label_counts[i] += np.random.laplace(0, beta, 1)
    new_label = np.argmax(label_counts)
    labels = np.append(labels, new_label)
  return labels

In [0]:
preds = eval_data(teachers, mnist_testset)


In [25]:
labels = mechanism(preds, epsilon)


(10000,)

In [0]:
#!pip install syft
from syft.frameworks.torch.differential_privacy import pate

In [28]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=labels, noise_eps=epsilon, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 1611.5129254649705
Data Dependent Epsilon: 34.607559293061705
